In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import numpy as np
import pandas as pd
import xgboost as xgb


In [2]:

dato = pd.read_csv('./Datos/prediccion/LaLiga_Matches_1995-2021.csv')

In [3]:
dato.shape

(10044, 10)

In [50]:
dato.head()

,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,categorical FTR,categorical Home,categorical Away,weekday
0,1995-96,1995-09-02,La Coruna,Valencia,3,0,H,2.0,0.0,H,2,21,41,5
1,1995-96,1995-09-02,Sp Gijon,Albacete,3,0,H,3.0,0.0,H,2,39,1,5
2,1995-96,1995-09-03,Ath Bilbao,Santander,4,0,H,2.0,0.0,H,2,3,36,6
3,1995-96,1995-09-03,Ath Madrid,Sociedad,4,1,H,1.0,1.0,D,2,4,38,6
4,1995-96,1995-09-03,Celta,Compostela,0,1,A,0.0,0.0,D,0,8,9,6


In [5]:
dato.isnull().sum()

Season      0
Date        0
HomeTeam    0
AwayTeam    0
FTHG        0
FTAG        0
FTR         0
HTHG        2
HTAG        2
HTR         2
dtype: int64

In [4]:
team_name = {}
index = 0
for idx, row in dato.iterrows():
    name = row['HomeTeam']
    if(name not in team_name.keys()):
        team_name[name] = index
        index += 1
    name = row['AwayTeam']
    if(name not in team_name.keys()):
        team_name[name] = index
        index += 1
        
team_name

{'La Coruna': 0,
 'Valencia': 1,
 'Sp Gijon': 2,
 'Albacete': 3,
 'Ath Bilbao': 4,
 'Santander': 5,
 'Ath Madrid': 6,
 'Sociedad': 7,
 'Celta': 8,
 'Compostela': 9,
 'Espanol': 10,
 'Salamanca': 11,
 'Merida': 12,
 'Betis': 13,
 'Sevilla': 14,
 'Tenerife': 15,
 'Valladolid': 16,
 'Barcelona': 17,
 'Vallecano': 18,
 'Real Madrid': 19,
 'Zaragoza': 20,
 'Oviedo': 21,
 'Hercules': 22,
 'Extremadura': 23,
 'Logrones': 24,
 'Mallorca': 25,
 'Alaves': 26,
 'Villareal': 27,
 'Malaga': 28,
 'Numancia': 29,
 'Las Palmas': 30,
 'Osasuna': 31,
 'Villarreal': 32,
 'Recreativo': 33,
 'Murcia': 34,
 'Levante': 35,
 'Getafe': 36,
 'Cadiz': 37,
 'Gimnastic': 38,
 'Almeria': 39,
 'Xerez': 40,
 'Granada': 41,
 'Elche': 42,
 'Eibar': 43,
 'Cordoba': 44,
 'Leganes': 45,
 'Girona': 46,
 'Huesca': 47}

In [9]:
n_matches = dato.shape[0]
n_features = dato.shape[1] - 1
n_homewins = len(dato[dato.FTR == 'H'])
win_rate = (float(n_homewins) / (n_matches)) * 100

print("Total number of matches: {}".format(n_matches))
print("Number of features: {}".format(n_features))
print("Number of matches won by home team: {}".format(n_homewins))
print("Win rate of home team: {:.2f}%".format(win_rate))

Total number of matches: 10044
Number of features: 9
Number of matches won by home team: 4772
Win rate of home team: 47.51%


In [5]:
lb_make_outs = LabelEncoder() # se le asigna un entero
dato['categorical FTR'] = lb_make_outs.fit_transform(dato['FTR'])
lb_make_teams = LabelEncoder()
dato['categorical Home'] = dato.HomeTeam.apply(lambda x: team_name[x])
dato['categorical Away'] = dato.AwayTeam.apply(lambda x: team_name[x])

In [189]:
lb_make_outs.transform(['A'])

array([0])

In [6]:
dato.Date = pd.to_datetime(dato.Date, dayfirst=True)
dato['weekday'] = dato.Date.dt.weekday #se puede acceder a la fecha

In [100]:
dato.weekday_cos

0       -0.222521
1       -0.222521
2        0.623490
3        0.623490
4        0.623490
           ...   
10039   -0.222521
10040   -0.222521
10041   -0.222521
10042    0.623490
10043    0.623490
Name: weekday_cos, Length: 10044, dtype: float64

In [7]:
dato['categorical FTR'].value_counts()

2    4772
0    2711
1    2561
Name: categorical FTR, dtype: int64

In [93]:
dato.head()

,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,categorical FTR,categorical Home,categorical Away,weekday,weekday_sin,weekday_cos
0,1995-96,1995-09-02,La Coruna,Valencia,3,0,H,2.0,0.0,H,2,21,41,5,-0.974928,-0.222521
1,1995-96,1995-09-02,Sp Gijon,Albacete,3,0,H,3.0,0.0,H,2,39,1,5,-0.974928,-0.222521
2,1995-96,1995-09-03,Ath Bilbao,Santander,4,0,H,2.0,0.0,H,2,3,36,6,-0.781831,0.623490
3,1995-96,1995-09-03,Ath Madrid,Sociedad,4,1,H,1.0,1.0,D,2,4,38,6,-0.781831,0.623490
4,1995-96,1995-09-03,Celta,Compostela,0,1,A,0.0,0.0,D,0,8,9,6,-0.781831,0.623490


In [8]:
columns_name=['weekday','categorical Home','categorical Away']
X_all = dato
y_all = dato['categorical FTR']

In [10]:
#Shuffle and split the dataset into training and testing set.
X_train, X_test, y_train, y_test = train_test_split( X_all, y_all, test_size=0.2, random_state=53)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (8035, 14) (8035,)
Test set: (2009, 14) (2009,)


In [11]:
model = xgb.XGBClassifier(n_estimators=4, max_depth=4)
model.fit(X_train[columns_name],y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=4, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [12]:
y_pred = model.predict(X_test[columns_name])
accuracy = np.mean(y_pred == y_test)
accuracy

0.5047287207565954

In [51]:

empates = lb_make_outs.inverse_transform(y_pred).tolist().count('D')
local = lb_make_outs.inverse_transform(y_pred).tolist().count('H')
visitante = lb_make_outs.inverse_transform(y_pred).tolist().count('A')
correctos = [elem1 for elem1, elem2 in zip(y_test, y_pred) if elem1 == elem2]
resultado = lb_make_outs.inverse_transform(correctos).tolist()

print('Empates: ', empates, ' Local: ', local, ' Visitante: ', visitante)
print('Empates acertados: ', resultado.count('D'), ' Local acertados: ', resultado.count('H'), ' Visitante acertados: ', resultado.count('A'))

Empates:  7  Local:  1779  Visitante:  223
Empates acertados:  1  Local acertados:  905  Visitante acertados:  108


In [13]:
def predict(dataframe, modelo=model):
  df = dataframe.copy()
  df.Date = pd.to_datetime(df.Date, dayfirst=True)
  df['weekday'] = df.Date.dt.weekday
  df['categorical Home'] = df.HomeTeam.apply(lambda x: team_name[x])
  df['categorical Away'] = df.AwayTeam.apply(lambda x: team_name[x])
  cols = ['weekday', 'categorical Home', 'categorical Away']
  predictions = model.predict(df[cols])
  return lb_make_outs.inverse_transform(predictions)

In [156]:
dato.columns

Index(['Season', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'categorical FTR', 'categorical Home',
       'categorical Away', 'weekday', 'weekday_sin', 'weekday_cos'],
      dtype='object')

In [166]:
drawinput = dato[dato.FTR == 'D'][[
    'Date',
    'HomeTeam',
    'AwayTeam'
]]

In [15]:
# HomeTeam, AwayTeam, FTR, pred
dato

,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,categorical FTR,categorical Home,categorical Away,weekday
0,1995-96,1995-09-02,La Coruna,Valencia,3,0,H,2.0,0.0,H,2,0,1,5
1,1995-96,1995-09-02,Sp Gijon,Albacete,3,0,H,3.0,0.0,H,2,2,3,5
2,1995-96,1995-09-03,Ath Bilbao,Santander,4,0,H,2.0,0.0,H,2,4,5,6
3,1995-96,1995-09-03,Ath Madrid,Sociedad,4,1,H,1.0,1.0,D,2,6,7,6
4,1995-96,1995-09-03,Celta,Compostela,0,1,A,0.0,0.0,D,0,8,9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10039,2020-21,2021-05-22,Osasuna,Sociedad,0,1,A,0.0,0.0,D,0,31,7,5
10040,2020-21,2021-05-22,Real Madrid,Villarreal,2,1,H,0.0,1.0,A,2,19,32,5
10041,2020-21,2021-05-22,Valladolid,Ath Madrid,1,2,A,1.0,0.0,H,0,16,6,5
10042,2020-21,2021-05-23,Granada,Getafe,0,0,D,0.0,0.0,D,1,41,36,6


In [61]:
model.save_model('./modelo0.json')